In [28]:
import time
from collections import deque

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist

import gym

from utils import ReplayBuffer,clip

In [29]:
device = 'cuda'

In [30]:
class Actor(nn.Module):
    def __init__(self, N_s, N_a, max_a):
        super().__init__()
        self.layer_1 = nn.Linear(N_s,200)
        self.layer_2 = nn.Linear(200,200)
        self.layer_3 = nn.Linear(200,N_a)
        self.N_s = N_s
        self.N_a = N_a
        self.max_a = torch.tensor(max_a, dtype = torch.float32).to(device)
    
    def forward(self, s):
        assert type(s) == torch.Tensor
        
        h = F.leaky_relu(self.layer_1(s))
        h = F.leaky_relu(self.layer_2(h))
        h = torch.tanh(self.layer_3(h)) * self. max_a
        return h

In [31]:
class Critic(nn.Module):
    def __init__(self,N_s,N_a):
        super().__init__()
        self.N_s = N_s
        self.N_a = N_a
        
        self.layer1 = nn.Linear(N_s+N_a,200)
        self.layer2 = nn.Linear(200,200)
        self.layer3 = nn.Linear(200,1)
    
    def forward(self,s,a):
        x = torch.cat((s,a),dim=-1)
        
        Q = F.leaky_relu(self.layer1(x))
        Q = F.leaky_relu(self.layer2(Q))
        Q = self.layer3(Q)
        
        return Q

In [32]:
class DDPG():
    def __init__(self, N_s, N_a, max_a):
        self.N_s = N_s
        self.N_a = N_a
        self.max_a = max_a
        
        self.actor = Actor(N_s, N_a, max_a)
        self.critic = Critic(N_s, N_a)
        self.target_critic = Critic(N_s, N_a)
        self.target_critic.load_state_dict(self.critic.state_dict())
        
        self.actor_optim = torch.optim.Adam(self.actor.parameters(), lr=1e-3)
        self.critic_optim = torch.optim.Adam(self.critic.parameters(), lr=1e-3)
        
    def to(self, device):
        self.actor.to(device)
        self.critic.to(device)
        self.target_critic.to(device)
        
    def polyak(self, tau):
        for param, target_param in zip(self.critic.parameters(),self.target_critic.parameters()):
            target_param.data.copy_(tau*param.data + (1-tau)*target_param.data)
    
    def target_Q(self, batch, gamma):
        NS = torch.tensor(batch['next_state'], dtype=torch.float32).to(device)
        R = torch.tensor(batch['reward'], dtype=torch.float32).to(device)
        D = torch.tensor(batch['done'],dtype=torch.int8).to(device)
        R = R.view(*R.shape,1)
        D = D.view(*D.shape,1)
        
        
        with torch.no_grad():
            A = self.actor(NS)
            NQ = self.target_critic(NS,A)
            
            return R + ( gamma * (~D) * NQ )
    
    def critic_loss(self, batch, gamma):
        S = torch.tensor(batch['state'], dtype=torch.float32).to(device)
        A = torch.tensor(batch['action'],dtype=torch.float32).to(device)
        
        Q = self.critic(S,A)
        target_Q = self.target_Q(batch, gamma)
        
        return F.mse_loss(Q,target_Q)
    
    def actor_loss(self, batch):
        S = torch.tensor(batch['state'], dtype=torch.float32).to(device)
        A = self.actor(S)
        #loss = -self.critic(S,A).mean()
        loss = -self.target_critic(S,A).mean()
        
        return loss
    
    def train_critic_once(self, batch, gamma=0.99, tau=0.005):
        self.critic_optim.zero_grad()
        L = self.critic_loss(batch, gamma)
        L.backward()
        self.critic_optim.step()
        
        self.polyak(tau)
        
        return L.item()
    
    def train_actor_once(self, batch):
        self.actor_optim.zero_grad()
        L = self.actor_loss(batch)
        L.backward()
        self.actor_optim.step()
        
        return L.item()
    
    def explore(self,s,sigma = 0.3):
        s = torch.tensor(s,dtype=torch.float32).to(device)
        with torch.no_grad():
            a = self.actor(s)
        a = a.detach().cpu().numpy()
        eps = np.random.normal(size = self.N_a, scale = sigma)
        
        return clip(a + eps,-self.max_a,self.max_a)

In [33]:
def run_episode(env, buffer, model, render = False, sigma = 0.3):
    s = env.reset()
    done = 0
    if render:
        env.render()
    while(not done):
        a = model.explore(s,sigma)
        ns, r, done, info = env.step(a)
        T = [s,ns,a,r,done]
        buffer.append(T)
        s = ns.copy()
        if render:
            env.render()

In [34]:
buffer = ReplayBuffer(max_len = 1e5)
env = gym.make('HalfCheetah-v2')

model = DDPG(17,6, np.array([1., 1., 1., 1., 1., 1., ]))
model.to(device)

In [35]:
# Add initial episodes to buffer w/o training
for episode in tqdm(range(3)):
    run_episode(env, buffer, model, render = False, sigma = 0.5)

100%|██████████| 3/3 [00:01<00:00,  2.84it/s]


In [36]:
# train
for episode in range(1000):
    render = (episode%30 == 0)
    run_episode(env, buffer, model, render = render, sigma = 0.3)
    
    for n in range(20):
        batch = buffer.sample(64)
        critic_L = model.train_critic_once(batch)
        if n%2 == 0:
            actor_L = model.train_actor_once(batch)
            print(f"critic loss: {critic_L}")
            print(f"actor loss: {actor_L}")
            

critic loss: 0.28720712661743164
actor loss: 0.2568731904029846
critic loss: 0.26839718222618103
actor loss: 0.19921834766864777
critic loss: 0.2989252209663391
actor loss: 0.19670307636260986
critic loss: 0.27499255537986755
actor loss: 0.17042189836502075
critic loss: 0.21138548851013184
actor loss: 0.1932041049003601
critic loss: 0.20215356349945068
actor loss: 0.18561962246894836
critic loss: 0.336820513010025
actor loss: 0.16610094904899597
critic loss: 0.21517662703990936
actor loss: 0.1770942658185959
critic loss: 0.32737797498703003
actor loss: 0.21814242005348206
critic loss: 0.20041225850582123
actor loss: 0.1885819137096405
critic loss: 0.21261729300022125
actor loss: 0.20145143568515778
critic loss: 0.1706506609916687
actor loss: 0.16598635911941528
critic loss: 0.1704578697681427
actor loss: 0.15043050050735474
critic loss: 0.22408708930015564
actor loss: 0.19503505527973175
critic loss: 0.17356698215007782
actor loss: 0.1740933358669281
critic loss: 0.17597603797912598
ac

critic loss: 0.028407983481884003
actor loss: 0.04146113246679306
critic loss: 0.020687909796833992
actor loss: -0.004398996010422707
critic loss: 0.031417056918144226
actor loss: 0.0443970188498497
critic loss: 0.03532952070236206
actor loss: 0.053513363003730774
critic loss: 0.022562311962246895
actor loss: -0.020372986793518066
critic loss: 0.02402665466070175
actor loss: 0.026809953153133392
critic loss: 0.02963927388191223
actor loss: -0.0027219653129577637
critic loss: 0.022590097039937973
actor loss: 0.03872513025999069
critic loss: 0.0349181666970253
actor loss: -0.05897269397974014
critic loss: 0.024339187890291214
actor loss: 0.04072897881269455
critic loss: 0.020313020795583725
actor loss: 0.012588007375597954
critic loss: 0.033292196691036224
actor loss: 0.013273593969643116
critic loss: 0.021288912743330002
actor loss: -0.002220064401626587
critic loss: 0.03317347913980484
actor loss: 0.10444781929254532
critic loss: 0.02054767683148384
actor loss: 0.03233528137207031
crit

critic loss: 0.029246743768453598
actor loss: 0.16684159636497498
critic loss: 0.0176417026668787
actor loss: 0.07240474224090576
critic loss: 0.021881571039557457
actor loss: 0.1131674274802208
critic loss: 0.016253773123025894
actor loss: 0.1259060651063919
critic loss: 0.013262457214295864
actor loss: 0.10920711606740952
critic loss: 0.021411443129181862
actor loss: 0.12322106957435608
critic loss: 0.013937722891569138
actor loss: 0.14031465351581573
critic loss: 0.02184877172112465
actor loss: 0.1080276221036911
critic loss: 0.010415395721793175
actor loss: 0.09547178447246552
critic loss: 0.01813937909901142
actor loss: 0.1004423052072525
critic loss: 0.025791611522436142
actor loss: 0.08517482876777649
critic loss: 0.017422717064619064
actor loss: 0.08598370850086212
critic loss: 0.013575146906077862
actor loss: 0.11506890505552292
critic loss: 0.011939771473407745
actor loss: 0.023108989000320435
critic loss: 0.009552063420414925
actor loss: 0.12183147668838501
critic loss: 0.01

critic loss: 0.007033245638012886
actor loss: 0.10860027372837067
critic loss: 0.008977201767265797
actor loss: 0.14456051588058472
critic loss: 0.014796123839914799
actor loss: 0.1418054699897766
critic loss: 0.011126548983156681
actor loss: 0.16028240323066711
critic loss: 0.01469759363681078
actor loss: 0.1119702160358429
critic loss: 0.012456128373742104
actor loss: 0.12102331221103668
critic loss: 0.008625539019703865
actor loss: 0.12669062614440918
critic loss: 0.008168144151568413
actor loss: 0.1438223123550415
critic loss: 0.022276621311903
actor loss: 0.12006644159555435
critic loss: 0.010580537840723991
actor loss: 0.16244181990623474
critic loss: 0.012792350724339485
actor loss: 0.13760191202163696
critic loss: 0.01366143673658371
actor loss: 0.08656105399131775
critic loss: 0.008379578590393066
actor loss: 0.1727626919746399
critic loss: 0.013542959466576576
actor loss: 0.10596280544996262
critic loss: 0.0088586937636137
actor loss: 0.14352299273014069
critic loss: 0.009254

critic loss: 0.0066070775501430035
actor loss: 0.19009089469909668
critic loss: 0.009568296372890472
actor loss: 0.10508624464273453
critic loss: 0.007826586253941059
actor loss: 0.14511562883853912
critic loss: 0.007031538523733616
actor loss: 0.1655738651752472
critic loss: 0.008990254253149033
actor loss: 0.12950152158737183
critic loss: 0.008290794678032398
actor loss: 0.140803724527359
critic loss: 0.011501245200634003
actor loss: 0.1331871747970581
critic loss: 0.014136831276118755
actor loss: 0.12039359658956528
critic loss: 0.005554533563554287
actor loss: 0.09821756184101105
critic loss: 0.013529608026146889
actor loss: 0.11717015504837036
critic loss: 0.010218803770840168
actor loss: 0.1768786460161209
critic loss: 0.007675092201679945
actor loss: 0.1418658345937729
critic loss: 0.008921021595597267
actor loss: 0.12058951705694199
critic loss: 0.010786904022097588
actor loss: 0.1440374255180359
critic loss: 0.006807185709476471
actor loss: 0.1490648239850998
critic loss: 0.00

...

critic loss: 0.0007006743107922375
actor loss: 0.01692952960729599
critic loss: 0.0009846335742622614
actor loss: 0.016544640064239502
critic loss: 0.0011149304918944836
actor loss: 0.015278400853276253
critic loss: 0.00042088187183253467
actor loss: 0.06767179071903229
critic loss: 0.0007294920505955815
actor loss: 0.025028564035892487
critic loss: 0.0011069260071963072
actor loss: 0.03321421518921852
critic loss: 0.0008495838847011328
actor loss: 0.030560432001948357
critic loss: 0.0003109285025857389
actor loss: 0.012437183409929276
critic loss: 0.0012164739891886711
actor loss: -0.009047901257872581
critic loss: 0.0010529108112677932
actor loss: 0.022526925429701805
critic loss: 0.0009746566647663713
actor loss: 0.06219101324677467
critic loss: 0.001563697587698698
actor loss: 0.026140619069337845
critic loss: 0.0002924835425801575
actor loss: 0.039059899747371674
critic loss: 0.0018056759145110846
actor loss: 0.010601876303553581
critic loss: 0.0007319044088944793
actor loss: 0.01

critic loss: 0.0008293632417917252
actor loss: 0.032786883413791656
critic loss: 0.0012597660534083843
actor loss: 0.018542464822530746
critic loss: 0.0007023644866421819
actor loss: 0.03197944164276123
critic loss: 0.0009052947862073779
actor loss: 0.03901413083076477
critic loss: 0.001627143006771803
actor loss: 0.020125137642025948
critic loss: 0.0006851301295682788
actor loss: 0.032717641443014145
critic loss: 0.0013802861794829369
actor loss: 0.048536576330661774
critic loss: 0.0007312186062335968
actor loss: 0.014774404466152191
critic loss: 0.0010262829018756747
actor loss: 0.004458250477910042
critic loss: 0.0006692857714369893
actor loss: 0.02581534907221794
critic loss: 0.0006372433854267001
actor loss: 0.0010393261909484863
critic loss: 0.000830454402603209
actor loss: 0.039463408291339874
critic loss: 0.0034511766862124205
actor loss: 0.0454399399459362
critic loss: 0.0005414844490587711
actor loss: 0.02128356508910656
critic loss: 0.0005219178856350482
actor loss: 0.022578

critic loss: 0.0004897452890872955
actor loss: 0.022159140557050705
critic loss: 0.0009643173543736339
actor loss: 0.007539363112300634
critic loss: 0.0002889908500947058
actor loss: 0.03917454555630684
critic loss: 0.0006644002860412002
actor loss: 0.0006953859701752663
critic loss: 0.0003495778364595026
actor loss: 0.061256833374500275
critic loss: 0.0005795780452899635
actor loss: 0.007712294347584248
critic loss: 0.0006169096450321376
actor loss: 0.03458733856678009
critic loss: 0.001088024815544486
actor loss: 0.01726105995476246
critic loss: 0.0006994590512476861
actor loss: 0.02984084188938141
critic loss: 0.0004945757682435215
actor loss: 0.032598793506622314
critic loss: 0.0009298513759858906
actor loss: 0.03953401744365692
critic loss: 0.0010530779836699367
actor loss: 0.03220588341355324
critic loss: 0.0006589366821572185
actor loss: 0.03606066107749939
critic loss: 0.0014303082134574652
actor loss: -0.01545975636690855
critic loss: 0.0012822350254282355
actor loss: 0.034197

critic loss: 0.0017277783481404185
actor loss: 0.01731090247631073
critic loss: 0.0005377844208851457
actor loss: 0.026911795139312744
critic loss: 0.0013874650467187166
actor loss: 0.02620280720293522
critic loss: 0.0009659478673711419
actor loss: 0.023707130923867226
critic loss: 0.000860815227497369
actor loss: 0.04731512814760208
critic loss: 0.0004923166707158089
actor loss: 0.008207942359149456
critic loss: 0.0009302862454205751
actor loss: 0.03979305922985077
critic loss: 0.0021674870513379574
actor loss: 0.06050892546772957
critic loss: 0.0020952315535396338
actor loss: 0.0460960790514946
critic loss: 0.0010074881138280034
actor loss: 0.06077881157398224
critic loss: 0.000871906231623143
actor loss: 0.025268953293561935
critic loss: 0.0005570622161030769
actor loss: 0.040640972554683685
critic loss: 0.0006682260427623987
actor loss: 0.04907979071140289
critic loss: 0.0013013017596676946
actor loss: -0.006419973447918892
critic loss: 0.000942403741646558
actor loss: 0.0074555221

critic loss: 0.0013521166983991861
actor loss: 0.039209045469760895
critic loss: 0.0012613050639629364
actor loss: 0.01851111836731434
critic loss: 0.0007479432970285416
actor loss: 0.03557481989264488
critic loss: 0.000747411628253758
actor loss: 0.026909489184617996
critic loss: 0.0014715425204485655
actor loss: 0.06958265602588654
critic loss: 0.001413810532540083
actor loss: 0.04063127189874649
critic loss: 0.000724626355804503
actor loss: 0.04002568498253822
critic loss: 0.0006433530943468213
actor loss: 0.019536178559064865
critic loss: 0.0007335948757827282
actor loss: 0.023034319281578064
critic loss: 0.0007045703823678195
actor loss: 0.044465601444244385
critic loss: 0.0005478312959894538
actor loss: 0.04435395449399948
critic loss: 0.0014786612009629607
actor loss: 0.016556723043322563
critic loss: 0.0005764039233326912
actor loss: 0.023647408932447433
critic loss: 0.00043830095091834664
actor loss: 0.02408161200582981
critic loss: 0.0022953879088163376
actor loss: 0.01607303

In [38]:
# test
for episode in range(1):
    run_episode(env, buffer, model, render = True, sigma = 0.0)

env.close()

Creating window glfw
